# 01 – Preprocessing of GSE113660 bulk RNA-seq counts

📓 This notebook:
- Loads the raw HTSeq count matrix from GEO (GSE113660_bulk_htseq_count.txt)
- Cleans and formats it into a gene × sample count matrix
- Constructs a sample metadata table with CD44 phenotype labels
- Saves both as CSVs under `data/processed/`:
  - `GSE113660_counts.csv`
  - `GSE113660_metadata.csv`


In [11]:
import pandas as pd
from pathlib import Path

# ---------------------------------------------------------------------
# Define base paths for the repository structure.
# Assumes this notebook is located in: notebooks/
# ---------------------------------------------------------------------
BASE_DIR = Path("..").resolve()               # assuming notebook is in notebooks/
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

DATA_RAW, DATA_PROCESSED


(PosixPath('/Users/melikagr/bioinformatics-projects/GSE113660-cd44-rnaseq-analysis/data/raw'),
 PosixPath('/Users/melikagr/bioinformatics-projects/GSE113660-cd44-rnaseq-analysis/data/processed'))

In [12]:
# ---------------------------------------------------------------------
# Load the raw HTSeq count file downloaded from GEO.
# This file contains:
# - A gene column (Unnamed: 0)
# - Gene type and ID columns (which we drop)
# - One column per sample (C1..C9)
# ---------------------------------------------------------------------
raw_path = DATA_RAW / "GSE113660_bulk_htseq_count.txt"

raw_df = pd.read_csv(raw_path, sep="\t")
raw_df.head()


,Unnamed: 0,geneType,numGeneId,SJRHB046412_C1,SJRHB046412_C2,SJRHB046412_C3,SJRHB046412_C4,SJRHB046412_C5,SJRHB046412_C6,SJRHB046412_C7,SJRHB046412_C8,SJRHB046412_C9
0,5S_rRNA,lincRNA,1,0,1,0,0,0,0,0,0,0
1,7SK,lincRNA,4,706,646,856,393,416,252,315,364,332
2,A1BG,protein_coding,1,41,107,42,20,47,45,42,24,36
3,A1BG-AS1,antisense,1,886,1028,1023,546,634,612,598,561,609
4,A1CF,protein_coding,1,0,0,0,0,0,0,1,0,0


In [13]:
# ---------------------------------------------------------------------
# Clean and format the raw dataframe:
# - Rename the gene column
# - Drop non-count columns
# - Set gene names as row index
# ---------------------------------------------------------------------
df = raw_df.copy()

# Examine columns for safety
print(raw_df.columns)

# Rename gene column
df = df.rename(columns={"Unnamed: 0": "gene"})

# Remove columns that are not read counts
df = df.drop(columns=["geneType", "numGeneId"])

# Set gene names as index -> final matrix shape: (genes × samples)
df = df.set_index("gene")

df.iloc[:5, :5]


Index(['Unnamed: 0', 'geneType', 'numGeneId', 'SJRHB046412_C1',
       'SJRHB046412_C2', 'SJRHB046412_C3', 'SJRHB046412_C4', 'SJRHB046412_C5',
       'SJRHB046412_C6', 'SJRHB046412_C7', 'SJRHB046412_C8', 'SJRHB046412_C9'],
      dtype='object')


,SJRHB046412_C1,SJRHB046412_C2,SJRHB046412_C3,SJRHB046412_C4,SJRHB046412_C5
gene,,,,,
5S_rRNA,0,1,0,0,0
7SK,706,646,856,393,416
A1BG,41,107,42,20,47
A1BG-AS1,886,1028,1023,546,634
A1CF,0,0,0,0,0


In [14]:
# ---------------------------------------------------------------------
# Save processed count matrix
# ---------------------------------------------------------------------
counts_path = DATA_PROCESSED / "GSE113660_counts.csv"
df.to_csv(counts_path)

counts_path, df.shape


(PosixPath('/Users/melikagr/bioinformatics-projects/GSE113660-cd44-rnaseq-analysis/data/processed/GSE113660_counts.csv'),
 (33572, 9))

In [15]:
# ---------------------------------------------------------------------
# Extract sample names from the dataframe.
# These are in the format: SJRHB046412_C1 ... SJRHB046412_C9
# ---------------------------------------------------------------------
sample_names = df.columns.tolist()
sample_names


['SJRHB046412_C1',
 'SJRHB046412_C2',
 'SJRHB046412_C3',
 'SJRHB046412_C4',
 'SJRHB046412_C5',
 'SJRHB046412_C6',
 'SJRHB046412_C7',
 'SJRHB046412_C8',
 'SJRHB046412_C9']

# Sample condition mapping
According to the GEO submission, CD44 phenotypes are defined based on FACS-sorted subpopulations:
- CD44_high: C2, C6, C7
- CD44_low:  C3, C4, C5
- unsorted:  C1, C8, C9

Dataset: GSE113660 (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE113660)


In [8]:
# ---------------------------------------------------------------------
# Map sample suffix (C1..C9) to biological condition.
#
# According to GEO description:
# - C2, C6, C7 are CD44_high
# - C3, C4, C5 are CD44_low
# - C1, C8, C9 are unsorted
#
# We construct a metadata table with columns:
#   sample | condition
# ---------------------------------------------------------------------
suffix_to_condition = {
    "C1": "unsorted",
    "C2": "CD44_high",
    "C3": "CD44_low",
    "C4": "CD44_low",
    "C5": "CD44_low",
    "C6": "CD44_high",
    "C7": "CD44_high",
    "C8": "unsorted",
    "C9": "unsorted",
}

rows = []
for s in sample_names:
    suffix = s.split("_")[-1]   # e.g. "C3"
    condition = suffix_to_condition.get(suffix)
    if condition is None:
        raise ValueError(f"No condition mapping for sample {s}")
    rows.append({"sample": s, "condition": condition})

metadata = pd.DataFrame(rows)
metadata

,sample,condition
0,SJRHB046412_C1,unsorted
1,SJRHB046412_C2,CD44_high
2,SJRHB046412_C3,CD44_low
3,SJRHB046412_C4,CD44_low
4,SJRHB046412_C5,CD44_low
5,SJRHB046412_C6,CD44_high
6,SJRHB046412_C7,CD44_high
7,SJRHB046412_C8,unsorted
8,SJRHB046412_C9,unsorted


In [9]:
# ---------------------------------------------------------------------
# Save metadata table
# ---------------------------------------------------------------------
metadata_path = DATA_PROCESSED / "GSE113660_metadata.csv"
metadata.to_csv(metadata_path, index=False)

metadata_path, metadata.shape

(PosixPath('/Users/melikagr/bioinformatics-projects/GSE113660-cd44-rnaseq-analysis/data/processed/GSE113660_metadata.csv'),
 (9, 2))

In [10]:
# ---------------------------------------------------------------------
# Validate alignment between processed counts and metadata.
# This ensures DESeq2, PCA, and ML steps will not fail due to mismatch.
# ---------------------------------------------------------------------
counts = pd.read_csv(counts_path, index_col=0)
metadata_loaded = pd.read_csv(metadata_path)

print(counts.shape)
print(metadata_loaded.shape)

print(counts.columns.tolist())
print(metadata_loaded["sample"].tolist())

# make sure order matches
assert list(counts.columns) == list(metadata_loaded["sample"]), "Sample mismatch between counts and metadata"


(33572, 9)
(9, 2)
['SJRHB046412_C1', 'SJRHB046412_C2', 'SJRHB046412_C3', 'SJRHB046412_C4', 'SJRHB046412_C5', 'SJRHB046412_C6', 'SJRHB046412_C7', 'SJRHB046412_C8', 'SJRHB046412_C9']
['SJRHB046412_C1', 'SJRHB046412_C2', 'SJRHB046412_C3', 'SJRHB046412_C4', 'SJRHB046412_C5', 'SJRHB046412_C6', 'SJRHB046412_C7', 'SJRHB046412_C8', 'SJRHB046412_C9']
